In [ ]:
# !pip install urllib3
# !pip install idna
# !pip install openpyxl

In [ ]:
# https://bipadportal.gov.np/api/v1/province/
# https://bipadportal.gov.np/api/v1/district/
# https://bipadportal.gov.np/api/v1/municipality/
# https://bipadportal.gov.np/api/v1/hazard/

import requests
import pandas as pd
def get_data_from_api(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes
        data = response.json()  # Convert response to JSON format
        return data
    except Exception as e:
        print("Error fetching data:", e)
        return None

In [ ]:
## HAZARD LIST
datas = get_data_from_api('https://bipadportal.gov.np/api/v1/hazard/')
natural_hazard_dataset=[]
non_natural_hazard_dataset=[]
if datas:
    for instance in datas['results']:
        if instance['type']=="natural":
            natural_hazard_dataset.append({
                "id":instance['id'],
                "title":instance['title'],
                "type":instance['type'],
            })
        else:
            non_natural_hazard_dataset.append({
                "id":instance['id'],
                "title":instance['title'],
                "type":instance['type'],
            })
        
else:
    print("empty data")
    
natural_hazard_dataset=sorted(natural_hazard_dataset, key=lambda x: x['id'])
non_natural_hazard_dataset=sorted(non_natural_hazard_dataset, key=lambda x: x['id'])
    
# print(natural_hazard_dataset)
# print(non_natural_hazard_dataset)

In [ ]:
[
    {'id': 2, 'title': 'Animal Incidents', 'type': 'natural'}, 
    {'id': 3, 'title': 'Avalanche', 'type': 'natural'}, 
    {'id': 6, 'title': 'Cold Wave', 'type': 'natural'}, 
    {'id': 8, 'title': 'Earthquake', 'type': 'natural'}, 
    {'id': 9, 'title': 'Epidemic', 'type': 'natural'}, 
    {'id': 10, 'title': 'Fire', 'type': 'natural'}, 
    {'id': 11, 'title': 'Flood', 'type': 'natural'}, 
    {'id': 12, 'title': 'Forest Fire', 'type': 'natural'}, 
    {'id': 13, 'title': 'Hailstorm', 'type': 'natural'}, 
    {'id': 14, 'title': 'Heavy Rainfall', 'type': 'natural'}, 
    {'id': 17, 'title': 'Landslide', 'type': 'natural'}, 
    {'id': 18, 'title': 'Other (Natural)', 'type': 'natural'}, 
    {'id': 19, 'title': 'Rainfall', 'type': 'natural'}, 
    {'id': 21, 'title': 'Snow Storm', 'type': 'natural'}, 
    {'id': 22, 'title': 'Storm', 'type': 'natural'}, 
    {'id': 23, 'title': 'Thunderbolt', 'type': 'natural'}, 
    {'id': 24, 'title': 'Wind Storm', 'type': 'natural'}, 
    {'id': 25, 'title': 'Drought', 'type': 'natural'}, 
    {'id': 26, 'title': 'Glacial lake outburst', 'type': 'natural'}, 
    {'id': 27, 'title': 'Heat wave', 'type': 'natural'}, 
    {'id': 28, 'title': 'Inundation', 'type': 'natural'}, 
    {'id': 29, 'title': 'Soil Erosion', 'type': 'natural'}, 
    {'id': 30, 'title': 'Volcanic eruption', 'type': 'natural'}, 
    {'id': 38, 'title': 'Famine', 'type': 'natural'}
]

#hazard_id and title dict

hazard_dict={
 2: 'Animal Incidents', 
 3: 'Avalanche', 
 6: 'Cold Wave', 
 8: 'Earthquake', 
 9: 'Epidemic', 
    10: 'Fire', 
    11: 'Flood', 
    12: 'Forest Fire', 
    13: 'Hailstorm', 
    14: 'Heavy Rainfall', 
    17: 'Landslide', 
    18: 'Other (Natural)', 
    19: 'Rainfall', 
    21: 'Snow Storm', 
    22: 'Storm', 
    23: 'Thunderbolt', 
    24: 'Wind Storm', 
    25: 'Drought', 
    26: 'Glacial lake outburst', 
    27: 'Heat wave', 
    28: 'Inundation', 
    29: 'Soil Erosion', 
    30: 'Volcanic eruption', 
    38: 'Famine', 'type': 'natural'}

In [ ]:

# https://bipadportal.gov.np/api/v1/incident/?rainBasin=&rainStation=&riverBasin=&riverStation=&hazard=12&incident_on__gt=2000-01-01T00%3A00%3A00%2B05%3A45&incident_on__lt=2024-05-31T23%3A59%3A59%2B05%3A45&expand=loss%2Cevent%2Cwards&ordering=-incident_on&limit=-1&data_source=drr_api

# https://bipadportal.gov.np/api/v1/incident/?hazard=12&incident_on__gt=2000-01-01T00%3A00%3A00%2B05%3A45&incident_on__lt=2024-05-31T23%3A59%3A59%2B05%3A45&ordering=incident_on&limit=-1

## FINAL CODE START

In [ ]:

# exclude_hazard_ids=[2,3,6,8,9,10,11,12,13,14,17,18,19,21,22,23,24,25,26,27,28,29,30,38] #original set
exclude_hazard_ids=[13,18,19,25,26,27,28,29,30,38]

# no_data=[13,18,19,25,26,27,28,29,30,38]

total_hazard_count=0

for hazard in natural_hazard_dataset:
    year_wise_incicent_counts_df=pd.DataFrame()
    hazard_id=hazard['id']
    try:
        if hazard_id in exclude_hazard_ids:
            continue
        print(f"*********{hazard_id}********")
        sheet_data={}
        year_wise_incident_count_sheet={}
        province_ids=[1,2,3,4,5,6,7]
        continue_check=False
        drop_column_list=['wards','titleNe','event','hazard','loss','createdBy','updatedBy','region','regionId','dataSourceId','dataSource','createdOn', 'modifiedOn','point']
        for province_id in province_ids:
            
            api_url=f"https://bipadportal.gov.np/api/v1/incident/?hazard={hazard_id}&province={province_id}&incident_on__gt=2000-01-01T00%3A00%3A00%2B05%3A45&incident_on__lt=2024-05-31T23%3A59%3A59%2B05%3A45&ordering=incident_on"
            datas = get_data_from_api(api_url)
            if datas:
                data_list=datas['results']
                if len(data_list)==0:
                    # print(province_id)
                    continue
                # print(data_list)
                total_hazard_count+=len(data_list)
                # print(len(data_list))
                df = pd.DataFrame(data_list)
                try:
                    df['latitude'] = df['point'].apply(lambda x: x['coordinates'][1])
                    df['longitude'] = df['point'].apply(lambda x: x['coordinates'][0])
                except Exception as e:
                    print("error lat lng",str(e))
                    continue_check=True
                    # continue
                try:
                    reported_on_dt = df['reportedOn'].str.split('T')
                    df['reportedOn'] = reported_on_dt.str[0]
                    df['reportedOnTime'] = reported_on_dt.str[1]
                except Exception as e:
                    print("error reported on",str(e))
                    continue_check=True
                    # continue
                try:
                    incident_on_dt = df['incidentOn'].str.split('T')
                    df['incidentOn'] = incident_on_dt.str[0]
                    df['incidentOnTime'] = incident_on_dt.str[1]
                    
                    df['incidentOn'] = pd.to_datetime(df['incidentOn'], format='%Y-%m-%d')
                    # Extract the year and create a new column 'year'
                    df['year'] = df['incidentOn'].dt.year
                except Exception as e:
                    print("error incident_on",str(e))
                    continue_check=True
                    # continue

                drop_column_list = [col for col in drop_column_list if col in df.columns]
                
                df.drop(drop_column_list, axis=1, inplace=True)

                try:
                    year_wise_incicent_counts_df = df.groupby('year').size()
                    
                    # Calculate the total occurrence of each unique year
                    year_counts = df['year'].value_counts().sort_index()

                    # Convert the result to a DataFrame
                    year_wise_incicent_counts_df = year_counts.reset_index()
                    year_wise_incicent_counts_df.columns = ['year', hazard['title']]
                    year_wise_incident_count_sheet[f"province_{province_id}"]=year_wise_incicent_counts_df
                except Exception as e:
                    print("error count",str(e))
                    continue_check=True
                    # continue
                
                sheet_data[f"province_{province_id}"]=df
        if continue_check==True:
            print("continue check true")
            # continue
        output_path=f"/Users/rabina/Projects/Bipad_data_extraction/output_1/{hazard['id']}_{hazard['title']}_provincewise.xlsx"
        output_path_2=f"/Users/rabina/Projects/Bipad_data_extraction/output_2/{hazard['id']}_{hazard['title']}_year_wise_incident_count.xlsx"
        
        # Create an Excel writer object
        if not df.empty:
            with pd.ExcelWriter(output_path) as writer:
                # Iterate over the dictionary and write each DataFrame to the corresponding sheet
                for sheet_name, data_frame in sheet_data.items():
                    data_frame.to_excel(writer, sheet_name=sheet_name, index=False)

        if not year_wise_incicent_counts_df.empty:
            with pd.ExcelWriter(output_path_2) as writer:
                # Iterate over the dictionary and write each DataFrame to the corresponding sheet
                for sheet_name, data_frame in year_wise_incident_count_sheet.items():
                    data_frame.to_excel(writer, sheet_name=sheet_name, index=False)
                
    except Exception as e:
        print(f"error in hazard id {hazard_id}: {str(e)}")
        continue

print("TOTAL HAZARD COUNT",total_hazard_count)
    

In [ ]:
#MERGING ALL HAZARDS
import os
import pandas as pd

directory = '/Users/rabina/Projects/Bipad_data_extraction/output_2'
files = os.listdir(directory)

# Initialize a dictionary to hold combined DataFrames for each province
combined_province_dfs = {f'province_{i}': pd.DataFrame() for i in range(1, 8)}
# print(combined_province_dfs)
sheet_names = list(combined_province_dfs.keys())

for file in files:
    hazard_id = int(file.split("_")[0])
    hazard_name = hazard_dict[hazard_id]

    file_path = os.path.join(directory, file)
    xls = pd.ExcelFile(file_path)

    for sheet_name in sheet_names:
        try:
            df = pd.read_excel(xls, sheet_name=sheet_name)
        except:
            continue
        combined_df = combined_province_dfs[sheet_name]

        if combined_df.empty:
            combined_province_dfs[sheet_name] = df
        else:
            if hazard_name not in combined_df.columns:
                combined_province_dfs[sheet_name] = pd.merge(combined_df, df, on='year', how='outer')

# Save the combined DataFrames to an Excel file
output_path_3 = "/Users/rabina/Projects/Bipad_data_extraction/provine_wise_hazard_combined_sheet.xlsx"

with pd.ExcelWriter(output_path_3) as writer:
    for sheet_name, data_frame in combined_province_dfs.items():
        data_frame.to_excel(writer, sheet_name=sheet_name, index=False)


## FINAL CODE END